# Answer Coalescence

Strider finds and ranks all paths that fulfill the query.  Some of these answers can be very similar.  For instance, two answers may differ only by a gene node, but perhaps those genes are from the same family.  This permits a higher-level, more parsimonious description of an answer set.

In [1]:
import requests
import json
from notebook_functions import reasonerGraphToCytoscape, knowledgeGraphToCytoscape
from cyjupyter import Cytoscape
import time

In [2]:

class Strider:
    def __init__(self):
        self.url='http://robokop.renci.org:5781'
    def call(self,question):
        message = {'message': {'query_graph': question}}
        return self.send_message(message)
    def send_message(self,message):
        response = requests.post(f'{self.url}/query',json=message)
        if response.status_code == 200:
            pid = response.json()
            return pid
        else:
            print(response.status_code)
            return None
    def query_result(self,pid):
        r = requests.get(f'{self.url}/results',params={'query_id':pid})
        return r.json()
    
strider = Strider()

In [30]:
def create_question(relation_type,disease_curie=None,chem_curie=None):
    question = { "nodes": [{"id":"n0", "type":"disease"},
                          {"id": "n1", "type":"chemical_substance"}],
                 "edges": [ {"id": "e0", "source_id": "n1", "target_id": "n0", "type": relation_type}]}
    if disease_curie is not None:
        question['nodes'][0]['curie']=disease_curie
    if chem_curie is not None:
        question['nodes'][1]['curie']=chem_curie
    return question

In [93]:
disease = "MONDO:0005148"   #T2D
edgetype="treats"
question = create_question(edgetype,disease_curie=disease)
message = {"message": {"query_graph": question}}

c=reasonerGraphToCytoscape(question)
Cytoscape(data=c, visual_style=c["style"], layout_name='grid', layout={"name": "grid", "height": "200px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'n0', 'label': 'disease\n[MONDO:0005148]', 'curie': 'MO…

In [94]:
p = strider.send_message(message)
print(p)

8bb416cd-9dfc-4e6f-adf8-b7677f6c821e


In [96]:
strider_result=strider.query_result(p)
while len(strider_result['results']) == 0:
    time.sleep(1)
    print('checking')
    strider_result = strider.query_result(p)
print('results found')

results found


In [99]:
for n in strider_result['knowledge_graph']['nodes']:
    print(n['id'], n['name'])
drugs = [n['id'] for n in strider_result['knowledge_graph']['nodes'] if n['id'].startswith('CHEBI')]

MONDO:0005148 type 2 diabetes mellitus
CHEBI:7986 Pentoxifylline
MESH:D000069450 Liraglutide
CHEBI:16325 lithocholic acid
CHEMBL.COMPOUND:CHEMBL1829174 FASIGLIFAM
PUBCHEM:3013166 L 783281
CHEBI:63631 topiramate
CHEBI:27999 tolbutamide
CHEMBL.COMPOUND:CHEMBL2146125 MAGNESIUM
CHEMBL.COMPOUND:CHEMBL259771 CHEMBL259771
CHEBI:50385 hemin
CHEBI:16731 (E)-cinnamaldehyde
CHEBI:4784 enalapril
CHEBI:6801 metformin
CHEBI:26513 quinolines
CHEBI:6682 mangiferin
PUBCHEM:23927 Lead
CHEBI:35457 EC 3.4.15.1 (peptidyl-dipeptidase A) inhibitor
MESH:D000077205 Pioglitazone
CHEBI:28088 genistein
CHEBI:30429 indigane
CHEBI:439329 (S)-fenfluramine
CHEBI:61016 angiotensin receptor antagonist
CHEBI:25681 omega-3 fatty acid
CHEBI:5384 glipizide
PUBCHEM:9947731 2-methyl-c-5-(4-(5-methyl-2-(4-methylphenyl)-4-oxazolyl)butyl)-1,3-dioxane-r-2-carboxylic acid
CHEBI:3650 chlorpropamide
CHEBI:73272 canagliflozin hydrate
CHEBI:50991 thiazolidenedione
CHEBI:5959 irbesartan
CHEBI:71271 saxagliptin hydrate
CHEBI:77148 gins

## Coalesce Answers by Property

In [133]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'property'}
property_result = requests.post(url,json=strider_result,params=params)
print(r.status_code)


200
results found
MONDO:0005148 type 2 diabetes mellitus
CHEBI:7986 Pentoxifylline
MESH:D000069450 Liraglutide
CHEBI:16325 lithocholic acid
CHEMBL.COMPOUND:CHEMBL1829174 FASIGLIFAM
PUBCHEM:3013166 L 783281
CHEBI:63631 topiramate
CHEBI:27999 tolbutamide
CHEMBL.COMPOUND:CHEMBL2146125 MAGNESIUM
CHEMBL.COMPOUND:CHEMBL259771 CHEMBL259771
CHEBI:50385 hemin
CHEBI:16731 (E)-cinnamaldehyde
CHEBI:4784 enalapril
CHEBI:6801 metformin
CHEBI:26513 quinolines
CHEBI:6682 mangiferin
PUBCHEM:23927 Lead
CHEBI:35457 EC 3.4.15.1 (peptidyl-dipeptidase A) inhibitor
MESH:D000077205 Pioglitazone
CHEBI:28088 genistein
CHEBI:30429 indigane
CHEBI:439329 (S)-fenfluramine
CHEBI:61016 angiotensin receptor antagonist
CHEBI:25681 omega-3 fatty acid
CHEBI:5384 glipizide
PUBCHEM:9947731 2-methyl-c-5-(4-(5-methyl-2-(4-methylphenyl)-4-oxazolyl)butyl)-1,3-dioxane-r-2-carboxylic acid
CHEBI:3650 chlorpropamide
CHEBI:73272 canagliflozin hydrate
CHEBI:50991 thiazolidenedione
CHEBI:5959 irbesartan
CHEBI:71271 saxagliptin hydrat

In [24]:
results = property_result.json()['results']
for result in results:
    chemnode = result['node_bindings'][1]
    print(len(chemnode['kg_id']), chemnode['p_values'], chemnode['properties'])

15 [8.072657013358605e-45, 6.483746222891755e-42] ['Drugs Used in Diabetes', 'Blood Glucose Lowering Agents']
14 [6.567843868028501e-43] ['Oral Hypoglycemics']
29 [3.270561921913802e-42] ['drugbank.approved']
16 [1.0840893279426186e-40] ['hypoglycemic_agent']
17 [6.27607905837926e-36] ['Alimentary Tract and Metabolism']
27 [1.3659039840862994e-35] ['therapeutic_flag']
25 [3.0865768196720107e-35] ['drugbank.investigational']
21 [5.086869670882937e-32] ['oral']
28 [8.588128044402654e-30, 1.2517181426225488e-29] ['drug', 'pharmaceutical']
30 [7.945602174143482e-28] ['application']
19 [2.0996850459555312e-23] ['inhibitor']
8 [5.756553307474046e-21] ['Hypoglycemia-Associated Agents']
12 [1.2015000966117868e-20] ['Enzyme Inhibitors']
11 [4.569813185172222e-19] ['Drugs that are Mainly Renally Excreted']
12 [1.3862102093088527e-18] ['EC_3.*_(hydrolase)_inhibitor']
35 [9.574538381340596e-18] ['biological_role']
15 [1.05887338692192e-17] ['enzyme_inhibitor']
5 [2.897881550528794e-17, 6.519080083

In [19]:
#modify notebook functions to show an answer.  Show something reasonably sized, like the first answer with 5 compounds.

## Coalesce Answers by Ontology

In [162]:
def create_pd_question(p_curie):
    question = { "nodes": [{"id":"n0", "type":"disease"},
                          {"id": "n1", "type":"phenotypic_feature","curie":p_curie}],
                 "edges": [ {"id": "e0", "source_id": "n1", "target_id": "n0", "type": "has_phenotype"}]}
    return question

In [ ]:
p_curie = "HP:0012649"
question = create_pd_question(p_curie)
message = {"message": {"query_graph": question}}
metformin_p = strider.send_message(message)
strider_result=strider.query_result(metformin_p)
while len(strider_result['results']) == 0:
    time.sleep(5)
    strider_result = strider.query_result(metformin_p)
print(len(strider_result['results']))

In [152]:
chem='CHEBI:28088'   #prednisone
edgetype="treats"
question = create_question(edgetype,chem_curie=chem)
message = {"message": {"query_graph": question}}

c=reasonerGraphToCytoscape(question)
Cytoscape(data=c, visual_style=c["style"], layout_name='grid', layout={"name": "grid", "height": "200px"})

Cytoscape(data={'elements': {'nodes': [{'data': {'id': 'n0', 'label': 'disease\n[]', 'curie': '', 'type': 'dis…

In [153]:
metformin_p = strider.send_message(message)
strider_result=strider.query_result(metformin_p)
while len(strider_result['results']) == 0:
    time.sleep(5)
    print('checking')
    strider_result = strider.query_result(metformin_p)
print('results found')

checking
results found


In [154]:
strider_result = strider.query_result(metformin_p)
for n in strider_result['knowledge_graph']['nodes']:
    print(n['id'], n['name'])

MONDO:0005280 prostatitis (disease)
CHEBI:28088 genistein
MONDO:0005148 type 2 diabetes mellitus
MONDO:0012883 acute promyelocytic leukemia


In [155]:
url = 'https://answercoalesce.renci.org/coalesce'
params = {'method':'ontology'}
ontology_result = requests.post(url,json=strider_result,params=params)
print(r.status_code)

200


In [156]:
results = ontology_result.json()['results']
print(len(results))

0
